## Model Detection Collection

Collects predicted bounding boxes of each trained model on each image of the test set for further evaluation

In [1]:
import pandas as pd
import pathlib
import os
import mmcv
from mmdet.apis import inference_detector, init_detector
import pathlib
from tqdm import tqdm
import torch

from mmrotate.registry import VISUALIZERS, DATASETS
from mmengine.config import Config

from mmrotate.utils import register_all_modules

/home/jose/miniconda3/envs/OCD/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH_IMAGES = '/home/jose/Programas/OCDDataset/mmrotate-1.x/data/OCD/test/images'
PATH_CONFIGS = '/home/jose/Programas/OCDDataset/mmrotate-1.x/configs/OCD-Cropped'
PATH_WORKDIRS = '/home/jose/Programas/OCDDataset/mmrotate-1.x/work_dirs'
PATH_OUTPUTS = '/home/jose/Programas/OCDDataset/detection_outputs'

### Inference

Collect configs for all trained models

In [3]:
list_models = []
for config in pathlib.Path(PATH_CONFIGS).iterdir():
    if config.is_dir():
        continue
    basename_no_ext = os.path.basename(config)[:-3]
    checkpoint = os.path.join(PATH_WORKDIRS, basename_no_ext, 'epoch_72.pth')
    if not os.path.exists(checkpoint):
        continue
    list_models.append((config, checkpoint))

print(f'Found {len(list_models)} trained models')

Found 13 trained models


In [4]:
%cd /home/jose/Programas/OCDDataset/mmrotate-1.x

/home/jose/Programas/OCDDataset/mmrotate-1.x


Load Ground Truth bounding boxes into a dataframe

In [5]:


config, _ = list_models[0]
cfg = Config.fromfile(config)
register_all_modules()
dataset = DATASETS.build(cfg.test_dataloader.dataset)

df_gt = []
for item in dataset:
    data_samples = item['data_samples']
    img_id = data_samples.img_id
    bboxes = data_samples.gt_instances.bboxes.tensor
    df = pd.DataFrame(bboxes, columns=['cx', 'cy', 'width', 'height', 'angle'])
    df.insert(0, 'evaluator', 'ground-truth')
    df.insert(0, 'file_name', img_id)
    df['detection_threshold'] = 1.0
    df_gt.append(df)

df_gt = pd.concat(df_gt)
df_gt


,file_name,evaluator,cx,cy,width,height,angle,detection_threshold
0,exp_M4_Mauricio_mcr5_frame_40,ground-truth,29.500000,247.000000,96.000000,59.000000,1.570796,1.0
1,exp_M4_Mauricio_mcr5_frame_40,ground-truth,104.396538,581.258667,122.377594,83.773506,1.165905,1.0
2,exp_M4_Mauricio_mcr5_frame_40,ground-truth,196.551376,641.218079,88.093498,149.375366,1.098167,1.0
3,exp_M4_Mauricio_mcr5_frame_40,ground-truth,399.000061,710.000061,143.676727,108.300522,0.501013,1.0
4,exp_M4_Mauricio_mcr5_frame_40,ground-truth,233.978607,826.720154,88.457916,144.678955,0.096918,1.0
...,...,...,...,...,...,...,...,...
83,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,ground-truth,422.000000,701.000000,46.076443,21.839451,0.287281,1.0
84,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,ground-truth,946.000000,179.000000,20.000000,22.000000,1.570796,1.0
85,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,ground-truth,50.000004,481.999969,161.664383,96.044884,1.545957,1.0
86,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,ground-truth,910.000000,436.500000,127.000000,96.000000,1.570796,1.0


Get results for each model

In [6]:
results = []
list_images = [os.path.join(PATH_IMAGES, s) + '.png' for s in df_gt['file_name'].unique()]

for config_file, checkpoint in list_models:
    print(f'Evaluating {config_file}')
    try:
        model = init_detector(config_file, checkpoint).to('cuda:0')

        model_name = os.path.basename(config_file)[:-3]
        out_dir = os.path.join(PATH_OUTPUTS, model_name)
        score_threshold = 0.5
        
        print(f'\tOutputs will be stored at {out_dir}')
        print(f'\tUsing score threshold={score_threshold}')

        visualizer = VISUALIZERS.build(model.cfg.visualizer)
        visualizer.dataset_meta = model.dataset_meta

        for image in tqdm(list_images):
            image_basename = os.path.basename(image)
            out_file = os.path.join(out_dir, image_basename)

            result = inference_detector(model, image)
            
            # Add detection results
            results.append((model_name, result))
            
            # Create visual output
            img = mmcv.imread(image)
            img = mmcv.imconvert(img, 'bgr', 'rgb')
            visualizer.add_datasample(
                'result',
                img,
                data_sample=result,
                draw_gt=False,
                show=False,
                wait_time=0,
                out_file=out_file,
                pred_score_thr=score_threshold)
    except Exception as e:
        print(e)
        print('-------------------------- Failed! --------------------------')

Evaluating /home/jose/Programas/OCDDataset/mmrotate-1.x/configs/OCD-Cropped/rotated-fcos-le90_r50_fpn_rr_1x_ocd-cropped.py
Loads checkpoint by local backend from path: /home/jose/Programas/OCDDataset/mmrotate-1.x/work_dirs/rotated-fcos-le90_r50_fpn_rr_1x_ocd-cropped/epoch_72.pth


/home/jose/miniconda3/envs/OCD/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


	Outputs will be stored at /home/jose/Programas/OCDDataset/mmrotate-1.x/detection_outputs/rotated-fcos-le90_r50_fpn_rr_1x_ocd-cropped
	Using score threshold=0.5


  0%|          | 0/30 [00:00<?, ?it/s]/home/jose/miniconda3/envs/OCD/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/jose/miniconda3/envs/OCD/lib/python3.8/site-packages/mmdet/visualization/palette.py:90: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  scales = 0.5 + (areas - min_area) // (max_area - min_area)
 10%|█         | 3/30 [00:02<00:17,  1.54it/s]/home/jo

Evaluating /home/jose/Programas/OCDDataset/mmrotate-1.x/configs/OCD-Cropped/redet-le90_re50_refpn_rr_1x_ocd-cropped.py
Please install e2cnn by "pip install -e git+https://github.com/QUVA-Lab/e2cnn.git#egg=e2cnn"
-------------------------- Failed! --------------------------
Evaluating /home/jose/Programas/OCDDataset/mmrotate-1.x/configs/OCD-Cropped/rotated-retinanet-rbox-le90_r50_fpn_rr_1x_ocd-cropped.py
Loads checkpoint by local backend from path: /home/jose/Programas/OCDDataset/mmrotate-1.x/work_dirs/rotated-retinanet-rbox-le90_r50_fpn_rr_1x_ocd-cropped/epoch_72.pth


...

100%|██████████| 30/30 [00:11<00:00,  2.50it/s]


Evaluating /home/jose/Programas/OCDDataset/mmrotate-1.x/configs/OCD-Cropped/s2anet-le90_r50_fpn_rr_1x_ocd-cropped.py
Loads checkpoint by local backend from path: /home/jose/Programas/OCDDataset/mmrotate-1.x/work_dirs/s2anet-le90_r50_fpn_rr_1x_ocd-cropped/epoch_72.pth
	Outputs will be stored at /home/jose/Programas/OCDDataset/mmrotate-1.x/detection_outputs/s2anet-le90_r50_fpn_rr_1x_ocd-cropped
	Using score threshold=0.5


  0%|          | 0/30 [00:00<?, ?it/s]/home/jose/miniconda3/envs/OCD/lib/python3.8/site-packages/mmdet/models/dense_heads/anchor_head.py:115: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '
100%|██████████| 30/30 [00:11<00:00,  2.54it/s]


Evaluating /home/jose/Programas/OCDDataset/mmrotate-1.x/configs/OCD-Cropped/rotated-retinanet-rbox-le90_r50_fpn_rr_probiou_1x_ocd-cropped.py
Loads checkpoint by local backend from path: /home/jose/Programas/OCDDataset/mmrotate-1.x/work_dirs/rotated-retinanet-rbox-le90_r50_fpn_rr_probiou_1x_ocd-cropped/epoch_72.pth
	Outputs will be stored at /home/jose/Programas/OCDDataset/mmrotate-1.x/detection_outputs/rotated-retinanet-rbox-le90_r50_fpn_rr_probiou_1x_ocd-cropped
	Using score threshold=0.5


100%|██████████| 30/30 [00:11<00:00,  2.70it/s]


### Dataframes

Add model detections on test set to a dataframe

In [7]:
preds_df = []
gt_df = None
# Get preds
for (model, result) in results:
    img = os.path.basename(result.img_path)[:-4]
    
    gt_bboxes = result.gt_instances
    preds = result.pred_instances
    preds_scores = torch.hstack([preds.bboxes, preds.scores.unsqueeze(-1)])
    pred_df = pd.DataFrame(preds_scores.cpu().numpy(), columns=['cx','cy','width','height','angle','detection_threshold'])
    pred_df.insert(0, 'evaluator', model)
    pred_df.insert(0, 'file_name', img)
    preds_df.append(pred_df)

preds_df = pd.concat(preds_df)
preds_df

,file_name,evaluator,cx,cy,width,height,angle,detection_threshold
0,exp_M4_Mauricio_mcr5_frame_40,rotated-fcos-le90_r50_fpn_rr_1x_ocd-cropped,403.262085,709.815979,111.755920,153.955566,-1.070023,0.805208
1,exp_M4_Mauricio_mcr5_frame_40,rotated-fcos-le90_r50_fpn_rr_1x_ocd-cropped,645.640625,376.847473,167.312164,89.596298,1.524949,0.788907
2,exp_M4_Mauricio_mcr5_frame_40,rotated-fcos-le90_r50_fpn_rr_1x_ocd-cropped,804.467957,327.393524,115.130814,153.067535,-1.523921,0.761100
3,exp_M4_Mauricio_mcr5_frame_40,rotated-fcos-le90_r50_fpn_rr_1x_ocd-cropped,1232.214233,857.436157,74.738045,329.111389,-0.641095,0.695975
4,exp_M4_Mauricio_mcr5_frame_40,rotated-fcos-le90_r50_fpn_rr_1x_ocd-cropped,964.851746,630.443909,56.655586,63.850662,1.436237,0.694953
...,...,...,...,...,...,...,...,...
120,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,rotated-retinanet-rbox-le90_r50_fpn_rr_probiou...,690.166504,673.584778,75.219444,36.250626,-0.793936,0.069284
121,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,rotated-retinanet-rbox-le90_r50_fpn_rr_probiou...,1034.217285,541.377197,86.621796,35.250435,-0.941018,0.067595
122,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,rotated-retinanet-rbox-le90_r50_fpn_rr_probiou...,1265.207031,48.047562,107.209198,45.721748,-1.465166,0.065436
123,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,rotated-retinanet-rbox-le90_r50_fpn_rr_probiou...,609.134705,544.097412,87.784660,34.443340,-0.980694,0.065334


Create a consolidated dataframe with Ground Truth and predicted bounding boxes

In [8]:
consolidated_df = pd.concat([df_gt, preds_df])
consolidated_df['type'] = 'test'
consolidated_df.to_csv(os.path.join(PATH_OUTPUTS, 'consolidated_df.csv'), index=False)

for evaluator_name, evaluator_group in consolidated_df.groupby('evaluator'):
    assert len(evaluator_group['file_name'].unique()) == len(list_images)

consolidated_df

,file_name,evaluator,cx,cy,width,height,angle,detection_threshold,type
0,exp_M4_Mauricio_mcr5_frame_40,ground-truth,29.500000,247.000000,96.000000,59.000000,1.570796,1.000000,test
1,exp_M4_Mauricio_mcr5_frame_40,ground-truth,104.396538,581.258667,122.377594,83.773506,1.165905,1.000000,test
2,exp_M4_Mauricio_mcr5_frame_40,ground-truth,196.551376,641.218079,88.093498,149.375366,1.098167,1.000000,test
3,exp_M4_Mauricio_mcr5_frame_40,ground-truth,399.000061,710.000061,143.676727,108.300522,0.501013,1.000000,test
4,exp_M4_Mauricio_mcr5_frame_40,ground-truth,233.978607,826.720154,88.457916,144.678955,0.096918,1.000000,test
...,...,...,...,...,...,...,...,...,...
120,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,rotated-retinanet-rbox-le90_r50_fpn_rr_probiou...,690.166504,673.584778,75.219444,36.250626,-0.793936,0.069284,test
121,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,rotated-retinanet-rbox-le90_r50_fpn_rr_probiou...,1034.217285,541.377197,86.621796,35.250435,-0.941018,0.067595,test
122,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,rotated-retinanet-rbox-le90_r50_fpn_rr_probiou...,1265.207031,48.047562,107.209198,45.721748,-1.465166,0.065436,test
123,exp_L1.4_Luana_A172_cloneF9(mito)+H2B_TMZ3hora...,rotated-retinanet-rbox-le90_r50_fpn_rr_probiou...,609.134705,544.097412,87.784660,34.443340,-0.980694,0.065334,test
